In [85]:
from datetime import date

import base64
import requests
import json

In [89]:
base_url = "https://playlist.cprnetwork.org/api/playlistCO/selectDate:{}"
# Dates are sent to the API in the following format yyyy-mm-dd (%Y-%m-%d) base64 encoded
june_dates = [date(2019, 6, i) for i in range(1, 31)]
july_dates = [date(2019, 7, i) for i in range(1, 26)]


def fetch_tracks(dates):
    tracks = []
    for d in dates:
        formatted = d.strftime('%Y-%m-%d').encode('utf-8')
        print(f'fetching {formatted}')
        encoded = base64.b64encode(formatted).decode('utf-8')
        url = base_url.format(str(encoded))
        response = requests.get(url)
        tracks.extend(response.json())
    return tracks
    
    


In [91]:
july_tracks = fetch_tracks(july_dates)

fetching b'2019-07-01'
fetching b'2019-07-02'
fetching b'2019-07-03'
fetching b'2019-07-04'
fetching b'2019-07-05'
fetching b'2019-07-06'
fetching b'2019-07-07'
fetching b'2019-07-08'
fetching b'2019-07-09'
fetching b'2019-07-10'
fetching b'2019-07-11'
fetching b'2019-07-12'
fetching b'2019-07-13'
fetching b'2019-07-14'
fetching b'2019-07-15'
fetching b'2019-07-16'
fetching b'2019-07-17'
fetching b'2019-07-18'
fetching b'2019-07-19'
fetching b'2019-07-20'
fetching b'2019-07-21'
fetching b'2019-07-22'
fetching b'2019-07-23'
fetching b'2019-07-24'
fetching b'2019-07-25'


In [77]:
def save_json(name, tracklist):
    # dont want to wait for the api again just save it
    with open(name, 'w') as f:
        f.write(json.dumps(tracklist))


In [92]:
print(f'june: {len(june_tracks)}')
print(f'july: {len(july_tracks)}')
save_json('june.json', june_tracks)
save_json('july.json', july_tracks)

june: 11388
july: 9639


In [94]:
spotify_url = "https://api.spotify.com/v1/search?q=album%3AHeartworms%20artist%3AThe%20Shins%20track%3A%20Name%20For%20You&type=track"
def spotify_search(track):
    # q=album%3AHeartworms%20artist%3AThe%20Shins%20track%3A%20Name%20For%20You&type=track
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer {}"
    }
    response = requests.get(spotify_url, headers=headers)
    return response.json()


In [95]:
spotify_search(None)

{'error': {'status': 401, 'message': 'The access token expired'}}